In [ ]:
import os
import pandas as pd
import numpy as np
from sdv.single_table import GaussianCopulaSynthesizer
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

In [ ]:
import os
import pandas as pd
import numpy as np
from sdv.single_table import GaussianCopulaSynthesizer
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

# 1. 데이터 로드 및 전처리 (이전 코드와 동일)
pd.set_option('display.max_columns', None)
path = '../../../../../archive/raw/HI-Small_Trans.csv'
df = pd.read_csv(path)


# 2. Timestamp 컬럼을 datetime 객체로 변환 (errors='coerce' 옵션 사용)
df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%Y/%m/%d %H:%M', errors='coerce')

# 3. 'Timestamp' 컬럼에서 NaT인 값 (변환 실패한 값)을 NaN으로 대체
df['Timestamp'] = df['Timestamp'].replace({pd.NaT: np.nan})

# 4. 결측치 처리 (NaN 값을 삭제 또는 대체)
df = df.dropna(subset=['Timestamp'])

# 5. DataFrame이 비어있는지 확인
if df.empty:
    print("DataFrame is empty after removing rows with missing Timestamp values.")
    # 이후 코드를 실행하지 않도록 처리 (예: return)
    exit()

# 6. datetime 객체를 Unix timestamp로 변환
df['Timestamp'] = df['Timestamp'].astype('int64') // 10**9

# (선택 사항) 필요에 따라 스케일링 또는 정규화를 수행
# 예: MinMaxScaler를 사용하여 0과 1 사이의 값으로 스케일링
scaler = MinMaxScaler()
df['Timestamp'] = scaler.fit_transform(df[['Timestamp']])

# 7. Object 타입 컬럼 원핫 인코딩 (대상 컬럼 줄이기)
# 예: 'Receiving Currency', 'Payment Currency', 'Payment Format' 컬럼만 원핫 인코딩
df = pd.get_dummies(df, columns=['Receiving Currency', 'Payment Currency', 'Payment Format'])

# 8. 'Account', 'Account.1' 컬럼은 라벨 인코딩
label_encoder = LabelEncoder()
df['Account'] = label_encoder.fit_transform(df['Account'])
df['Account.1'] = label_encoder.fit_transform(df['Account.1'])

# 9. 로그 변환
# 예시: 'Amount Received', 'Amount Paid' 컬럼에 로그 변환 적용
log_cols = ['Amount Received', 'Amount Paid']
for col in log_cols:
    df[col] = np.log1p(df[col])

# 10. 수치형 데이터만 선택
df_numeric = df.select_dtypes(include=[np.number])

# 11. NaN 값 처리 (0으로 대체)
df_numeric = df_numeric.fillna(0)

# 12. 무한대 (inf) 값 처리 (최대값으로 대체)
df_numeric = df_numeric.replace([np.inf, -np.inf], np.nan)
df_numeric = df_numeric.fillna(df_numeric.max())

# 13. 통계 정보 추출
means = df_numeric.mean()
stds = df_numeric.std()
correlations = df_numeric.corr()

# 14. 통계 정보 기반 DataFrame 생성
statistical_data = pd.DataFrame({
    'mean': means,
    'std': stds
})

# 15. GaussianCopula 모델 학습 (통계 정보 사용)
metadata = {
    "fields": {
        col: {"type": "numerical"} for col in df_numeric.columns
    }
}

model = GaussianCopulaSynthesizer(metadata=metadata)
model.fit(statistical_data) # statistical_data 를 fit에 넣어줍니다.

# 16. 합성 데이터 생성
synthetic_data = model.sample(num_rows=1000)
synthetic_df = pd.DataFrame(synthetic_data, columns=df_numeric.columns)

# 17. 역변환
# 예시: 로그 변환된 컬럼에 대해 역변환 적용
for col in log_cols:
    synthetic_df[col] = np.expm1(synthetic_df[col])

# 18. 합성 데이터 출력 (선택 사항)
print(synthetic_df.head())

# 19. 품질 평가 (SDV 활용)
from sdv.evaluation import evaluate
quality_report = evaluate(df, synthetic_df, metrics=['KSTest']) # 원본 df 와 비교
print(quality_report)

AttributeError: 'dict' object has no attribute 'validate'

In [ ]:
import pandas as pd
import numpy as np
from sdv.single_table import GaussianCopulaSynthesizer
from sdv.metadata import SingleTableMetadata # SingleTableMetadata 임포트
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

# 1. 데이터 로드 및 전처리 (이전 코드와 동일)
# 예시: df = pd.read_csv('your_data.csv') # 실제 데이터 파일 경로로 변경

# 2. Timestamp 컬럼을 datetime 객체로 변환 (errors='coerce' 옵션 사용)
df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%Y/%m/%d %H:%M', errors='coerce')

# 3. 'Timestamp' 컬럼에서 NaT인 값 (변환 실패한 값)을 NaN으로 대체
df['Timestamp'] = df['Timestamp'].replace({pd.NaT: np.nan})

# 4. 결측치 처리 (NaN 값을 삭제 또는 대체)
df = df.dropna(subset=['Timestamp'])

# 5. DataFrame이 비어있는지 확인
if df.empty:
    print("DataFrame is empty after removing rows with missing Timestamp values.")
    # 이후 코드를 실행하지 않도록 처리 (예: return)
    exit()

# 6. datetime 객체를 Unix timestamp로 변환
df['Timestamp'] = df['Timestamp'].astype('int64') // 10**9

# (선택 사항) 필요에 따라 스케일링 또는 정규화를 수행
# 예: MinMaxScaler를 사용하여 0과 1 사이의 값으로 스케일링
scaler = MinMaxScaler()
df['Timestamp'] = scaler.fit_transform(df[['Timestamp']])

# 7. Object 타입 컬럼 원핫 인코딩 (대상 컬럼 줄이기)
# 예: 'Receiving Currency', 'Payment Currency', 'Payment Format' 컬럼만 원핫 인코딩
df = pd.get_dummies(df, columns=['Receiving Currency', 'Payment Currency', 'Payment Format'])

# 8. 'Account', 'Account.1' 컬럼은 라벨 인코딩
label_encoder = LabelEncoder()
df['Account'] = label_encoder.fit_transform(df['Account'])
df['Account.1'] = label_encoder.fit_transform(df['Account.1'])

# 9. 로그 변환
# 예시: 'Amount Received', 'Amount Paid' 컬럼에 로그 변환 적용
log_cols = ['Amount Received', 'Amount Paid']
for col in log_cols:
    df[col] = np.log1p(df[col])

# 10. 수치형 데이터만 선택
df_numeric = df.select_dtypes(include=[np.number])

# 11. NaN 값 처리 (0으로 대체)
df_numeric = df_numeric.fillna(0)

# 12. 무한대 (inf) 값 처리 (최대값으로 대체)
df_numeric = df_numeric.replace([np.inf, -np.inf], np.nan)
df_numeric = df_numeric.fillna(df_numeric.max())

# 13. 통계 정보 추출
means = df_numeric.mean()
stds = df_numeric.std()
correlations = df_numeric.corr()

# 14. 통계 정보 기반 DataFrame 생성
statistical_data = pd.DataFrame({
    'mean': means,
    'std': stds
})

# 15. GaussianCopula 모델 학습 (통계 정보 사용)
metadata = {
    "fields": {
        col: {"type": "numerical"} for col in df_numeric.columns
    }
}

metadata = SingleTableMetadata(metadata) # SingleTableMetadata 객체 생성
model = GaussianCopulaSynthesizer(metadata=metadata)
model.fit(df_numeric) # statistical_data 가 아닌 df_numeric 을 넣어줘야 함

# 16. 합성 데이터 생성
synthetic_data = model.sample(num_rows=1000)
synthetic_df = pd.DataFrame(synthetic_data, columns=df_numeric.columns)

# 17. 역변환
# 예시: 로그 변환된 컬럼에 대해 역변환 적용
for col in log_cols:
    synthetic_df[col] = np.expm1(synthetic_df[col])

# 18. 합성 데이터 출력 (선택 사항)
print(synthetic_df.head())

# 19. 품질 평가 (SDV 활용)
from sdv.evaluation import evaluate
quality_report = evaluate(df, synthetic_df, metrics=['KSTest']) # 원본 df 와 비교
print(quality_report)

In [8]:
df_numeric

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Amount Paid,Is Laundering,Receiving Currency_Australian Dollar,Receiving Currency_Bitcoin,Receiving Currency_Brazil Real,Receiving Currency_Canadian Dollar,Receiving Currency_Euro,Receiving Currency_Mexican Peso,Receiving Currency_Ruble,Receiving Currency_Rupee,Receiving Currency_Saudi Riyal,Receiving Currency_Shekel,Receiving Currency_Swiss Franc,Receiving Currency_UK Pound,Receiving Currency_US Dollar,Receiving Currency_Yen,Receiving Currency_Yuan,Payment Currency_Australian Dollar,Payment Currency_Bitcoin,Payment Currency_Brazil Real,Payment Currency_Canadian Dollar,Payment Currency_Euro,Payment Currency_Mexican Peso,Payment Currency_Ruble,Payment Currency_Rupee,Payment Currency_Saudi Riyal,Payment Currency_Shekel,Payment Currency_Swiss Franc,Payment Currency_UK Pound,Payment Currency_US Dollar,Payment Currency_Yen,Payment Currency_Yuan,Payment Format_ACH,Payment Format_Bitcoin,Payment Format_Cash,Payment Format_Cheque,Payment Format_Credit Card,Payment Format_Reinvestment,Payment Format_Wire
0,0.000786,10,942,10,900,8.215639,8.215639,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
1,0.000786,3208,990,1,949,0.009950,0.009950,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
2,0.000000,3209,991,3209,940,9.594008,9.594008,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
3,0.000079,12,997,12,945,7.940217,7.940217,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
4,0.000236,10,999,10,947,10.510095,10.510095,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5078340,0.565520,54219,495979,256398,419726,0.144081,0.144081,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
5078341,0.564656,15,495988,256398,419726,0.102672,0.102672,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
5078342,0.565323,154365,495980,256398,419726,0.004976,0.004976,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
5078343,0.565088,256398,495978,256398,419726,0.037697,0.037697,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [ ]:
import os
import pandas as pd
import numpy as np
from sdv.single_table import GaussianCopulaSynthesizer
from sdv.metadata import SingleTableMetadata
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

# 1. 데이터 로드 및 전처리 (이전 코드와 동일)
pd.set_option('display.max_columns', None)
path = '../../../../../archive/raw/HI-Small_Trans.csv'
df = pd.read_csv(path)


# 2. Timestamp 컬럼을 datetime 객체로 변환 (errors='coerce' 옵션 사용)
df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%Y/%m/%d %H:%M', errors='coerce')

# 3. 'Timestamp' 컬럼에서 NaT인 값 (변환 실패한 값)을 NaN으로 대체
df['Timestamp'] = df['Timestamp'].replace({pd.NaT: np.nan})

# 4. 결측치 처리 (NaN 값을 삭제 또는 대체)
df = df.dropna(subset=['Timestamp'])

# 5. DataFrame이 비어있는지 확인
if df.empty:
    print("DataFrame is empty after removing rows with missing Timestamp values.")
    # 이후 코드를 실행하지 않도록 처리 (예: return)
    exit()

# 6. datetime 객체를 Unix timestamp로 변환
df['Timestamp'] = df['Timestamp'].astype('int64') // 10**9

# (선택 사항) 필요에 따라 스케일링 또는 정규화를 수행
# 예: MinMaxScaler를 사용하여 0과 1 사이의 값으로 스케일링
scaler = MinMaxScaler()
df['Timestamp'] = scaler.fit_transform(df[['Timestamp']])

# 7. Object 타입 컬럼 원핫 인코딩 (대상 컬럼 줄이기)
# 예: 'Receiving Currency', 'Payment Currency', 'Payment Format' 컬럼만 원핫 인코딩
df = pd.get_dummies(df, columns=['Receiving Currency', 'Payment Currency', 'Payment Format'])

# 8. 'Account', 'Account.1' 컬럼은 라벨 인코딩
label_encoder = LabelEncoder()
df['Account'] = label_encoder.fit_transform(df['Account'])
df['Account.1'] = label_encoder.fit_transform(df['Account.1'])

# 9. 로그 변환
# 예시: 'Amount Received', 'Amount Paid' 컬럼에 로그 변환 적용
log_cols = ['Amount Received', 'Amount Paid']
for col in log_cols:
    df[col] = np.log1p(df[col])

# 10. 수치형 데이터만 선택
df_numeric = df.select_dtypes(include=[np.number])

# 11. NaN 값 처리 (0으로 대체)
df_numeric = df_numeric.fillna(0)

# 12. 무한대 (inf) 값 처리 (최대값으로 대체)
df_numeric = df_numeric.replace([np.inf, -np.inf], np.nan)
df_numeric = df_numeric.fillna(df_numeric.max())

# 13. 통계 정보 추출 (사용하지 않음)
# means = df_numeric.mean()
# stds = df_numeric.std()
# correlations = df_numeric.corr()

# 14. 통계 정보 기반 DataFrame 생성 (사용하지 않음)
# statistical_data = pd.DataFrame({
#     'mean': means,
#     'std': stds
# })

# 15. GaussianCopula 모델 학습 (df_numeric 사용)
metadata = SingleTableMetadata.from_dataframe(df_numeric)

model = GaussianCopulaSynthesizer(metadata=metadata)
model.fit(df_numeric) # statistical_data 가 아닌 df_numeric 을 fit에 넣어줘야 함

# 16. 합성 데이터 생성
synthetic_data = model.sample(num_rows=1000)
synthetic_df = pd.DataFrame(synthetic_data, columns=df_numeric.columns)

# 17. 역변환
# 예시: 로그 변환된 컬럼에 대해 역변환 적용
for col in log_cols:
    synthetic_df[col] = np.expm1(synthetic_df[col])

# 18. 합성 데이터 출력 (선택 사항)
print(synthetic_df.head())

# 19. 품질 평가 (SDV 활용)
from sdv.evaluation import evaluate
quality_report = evaluate(df, synthetic_df, metrics=['KSTest']) # 원본 df 와 비교
print(quality_report)